In [3]:
import numpy as np
import cv2 as cv2
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import math

In [2]:
R_lst = []
theta_lst = np.array([-90, 0, 90, 180], np.float) / 180 * np.pi
phi_lst = np.array([90, -90], np.float) / 180 * np.pi

In [3]:
print("angle_axis for theta")
for theta in theta_lst:
    angle_axis = theta * np.array([0, 1, 0], np.float)
    print(angle_axis)
    R = cv2.Rodrigues(angle_axis)[0]
    R_lst.append(R)
    
print("angle_axis for phi")
for phi in phi_lst:
    angle_axis = phi * np.array([1, 0, 0], np.float)
    print(angle_axis)
    R = cv2.Rodrigues(angle_axis)[0]
    R_lst.append(R)
    
len(R_lst)

angle_axis for theta
[-0.         -1.57079633 -0.        ]
[0. 0. 0.]
[0.         1.57079633 0.        ]
[0.         3.14159265 0.        ]
angle_axis for phi
[1.57079633 0.         0.        ]
[-1.57079633 -0.         -0.        ]


6

In [4]:
R_lst = [Variable(torch.FloatTensor(x)) for x in R_lst]
len(R_lst)

6

In [6]:
output_w = 1024
theta_start = math.pi - (math.pi / output_w)
theta_end = -math.pi
theta_step = 2 * math.pi / output_w
theta_range = torch.arange(theta_start, theta_end, -theta_step)

In [10]:
len(theta_range)

1024

In [11]:
output_h = 512
phi_start = 0.5 * math.pi - (0.5 * math.pi / output_h)
phi_end = -0.5 * math.pi
phi_step = math.pi / output_h
phi_range = torch.arange(phi_start, phi_end, -phi_step)

In [20]:
len(phi_range)

512

In [14]:
theta_map = theta_range.unsqueeze(0).repeat(output_h, 1)
phi_map = phi_range.unsqueeze(-1).repeat(1, output_w)
lonlat_map = torch.stack([theta_map, phi_map], dim=-1)

In [16]:
theta_map.shape

torch.Size([512, 1024])

In [17]:
phi_map.shape

torch.Size([512, 1024])

In [21]:
lonlat_map.shape

torch.Size([512, 1024, 2])

In [25]:
t = torch.rand(4, 4)
b = t.view(2, 8,1)

In [27]:
b.shape

torch.Size([2, 8, 1])

In [28]:
t

tensor([[0.0030, 0.6587, 0.5860, 0.6310],
        [0.3428, 0.0345, 0.3218, 0.6838],
        [0.2435, 0.3161, 0.8659, 0.4954],
        [0.3181, 0.7337, 0.8286, 0.2703]])

In [30]:
radius = 128
x_3d = (radius * torch.cos(phi_map) * torch.sin(theta_map)).view(output_h, output_w, 1)
y_3d = (radius * torch.sin(phi_map)).view(output_h, output_w, 1)
z_3d = (radius * torch.cos(phi_map) * torch.cos(theta_map)).view(output_h, output_w, 1)

In [32]:
x_3d.shape

torch.Size([512, 1024, 1])

In [33]:
grid_ball = torch.cat([x_3d, y_3d, z_3d], 2).view(output_h, output_w, 3)

In [35]:
grid_ball.shape

torch.Size([512, 1024, 3])

In [36]:
# Compute the down grid
radius_ratio_down = torch.abs(y_3d / radius)
grid_down_raw = grid_ball / radius_ratio_down.view(output_h, output_w, 1).expand(-1, -1, 3)
grid_down_w = (-grid_down_raw[:, :, 0].clone() / radius).unsqueeze(-1)
grid_down_h = (-grid_down_raw[:, :, 2].clone() / radius).unsqueeze(-1)
grid_down = torch.cat([grid_down_w, grid_down_h], 2).unsqueeze(0)
mask_down = (((grid_down_w <= 1) * (grid_down_w >= -1)) * ((grid_down_h <= 1) * (grid_down_h >= -1)) *
            (grid_down_raw[:, :, 1] == -radius).unsqueeze(2)).float()

In [43]:
grid_down_h.shape

torch.Size([512, 1024, 1])

In [45]:
radius_ratio_up = torch.abs(y_3d / radius)
grid_up_raw = grid_ball / radius_ratio_up.view(output_h, output_w, 1).expand(-1, -1, 3)
grid_up_w = (-grid_up_raw[:, :, 0].clone() / radius).unsqueeze(-1)
grid_up_h = (grid_up_raw[:, :, 2].clone() / radius).unsqueeze(-1)
grid_up = torch.cat([grid_up_w, grid_up_h], 2).unsqueeze(0)
mask_up = (((grid_up_w <= 1) * (grid_up_w >= -1)) * ((grid_up_h <= 1) * (grid_up_h >= -1)) *
          (grid_up_raw[:, :, 1] == radius).unsqueeze(2)).float()

In [54]:
mask_up.shape

torch.Size([512, 1024, 1])